This error is from the toolbox of MT5: {} it has a CSV format of "Description, File, Line, Column," columns with their corresponding rows.  But I only want you to care about what is written in the Description and Line because that's the 'error' it wants to depict. Then, after that, revise the previous code you've given me based on that 'Description' withthe  corresponding 'Line' error specification statements.